In [ ]:
import string
import configparser
import pandas as pd
import boto3
from io import StringIO
from nba_api.stats.static import teams
from nba_api.stats.static import players

## Baller-Metrics

### Introduction

If you ask me, one of the coolest appilications of Data Science is in sports, asides summary statistics, the field of Sport Analytics deals with the analysis of sport data to reveal insightful patterns that can improve in-game performance, reduce uncertainty in the outcome of games or even uncover overlooked talent. The techniques behind sport analytics are some of the most elegant implementations of mathematical modelling, however, the results are the stuff of headlines.

In this project you will journey with me as we curate a robust dataset that will allow for a number of sport analytics techniques. The dataset will be primarily based on game-by-game data for each active player. Information on each team and player attributes will be combined to this dataset to form a database.

The project follows the follow steps:
* Step 1: Scrape Data
* Step 2: Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: conclusions and recommendation

### Scrape Data

In the `baller_metrics_scrapper.py` script, we scrape items from the html table containing the dataset into a dictionary called `Player Stats`. We then convert the dictionary into a dataframe and finally read this dataframe into a csv file.

#### Description of Data Sources

| Data Set | Format | Description |
| ---      | ---    | ---         |
|[Game by game data for each player](https://www.basketball-reference.com/players/)| Web pages(html)| The dataset contains  game-by-game data for each active players, freely curated and hosted by ***basketball-reference.com***. Total amount of stats wold make up well over 1 million rows scattered across thousands of pages.|
|[Dataset of teams](https://github.com/swar/nba_api)| API | This dataset is from `nba-api` library. The library contains endpoints with updated data and simplied functions for retrieving them|
|[Dataset of players](https://github.com/swar/nba_api)| API| From Same end point above, but contains attribute information on tables.|

### Assess Data

Here we have to clean the data. To make the dataset sparkly we folow these steps:
1. To add column names
2. Remove rows where players did not play, and rows that only contain missing values
2. Provide a data dictionary outside this notebook. 


In [ ]:
player_stat = pd.read_csv("player_stat.csv")

##### Understanding and cleaning

Before reading the dataframe into a database table, we would like to understanding the nature of the data and some of its percularities. In this our case we want to understand why some values are missing and we want to ensure that most of our dataset contains relevant information.


In [ ]:
# View data types on all tables
player_stat.info()


The scrapper encountered a table differeny from others this table contains LeBron's highschool stats we can entries from this table by filtering for rows without a date i the date column

In [ ]:
player_stat = player_stat.iloc[:, 1:]
player_stat = player_stat[player_stat.Date.str.match("^[0-9]+")]
 

In [ ]:
player_stat["Date"] = pd.to_datetime(player_stat["Date"], format='%Y-%m-%d')

It seems the `GS` also encodes columns where values are missing as  ***Not With Team*** or ***Did Not Dress***

In [ ]:
player_stat[player_stat.FG.isna()].iloc[10:15, 10:]

In [ ]:
player_stat = player_stat[~((player_stat.GS == "Did Not Dress") | (player_stat.GS == "Not With Team"))]

In [ ]:
player_stat = player_stat[~((player_stat.MP == "Did Not Dress") | (player_stat.MP == "Not With Team"))]

Players sometimes get suspended counting the amount of games each player is suspended could be an indicator of each players could be an indicator of a players discliplinary record or play style. For this reason we can create a seperate table for those records and/or add them to the table where we store player attributes.

**to do:** create a running total of suspensions in the fact table.

In [ ]:
player_suspensions = player_stat[((player_stat.GS == "Player Suspended") | (player_stat.MP == "Player Suspended"))]

In [ ]:
player_stat = player_stat[~((player_stat.GS == "Player Suspended") | (player_stat.MP == "Player Suspended"))]

In [ ]:
player_stat.info()#iloc[:, 10:]

Now that we have removed rows where the players didn't play we can now begin to examine the nature of missing in the data. My hunch is that some missing values represent stats that may be zeros.

We will investigate using the code below.

In [ ]:
player_stat[player_stat.three_pct == 0.0].iloc[10:50, 10:] #Zeros exist

In [ ]:
player_stat[player_stat.three_pct.isna()].iloc[10:50, 10:]

Here we can see that the `three_pct` and `FG_pct` which represent **three point percentage** and **Field goal percentage**. Since zeros exist in the data set this missing values likely mean that the percentage values are **undefined** because the player didn't attempt any shot. 

In [ ]:
player_stat[player_stat.plus_minus.isna()].iloc[:, 10:]

+/- minus scores is a measure how impactful a player has been. Rows where players played less than 1 minute. Seem to be missing. Since these rows carry no information we can exclude them.

In [ ]:
player_stat = player_stat[~(player_stat.plus_minus.isna())]
player_stat.to_csv('player_stats.csv')



##### Attribute Tables

Here we will source for the attribute columns sourcing data from the `nba-api`

In [ ]:
teams = pd.DataFrame(teams.get_teams())

teams = teams.drop('id', axis = 1)

In [ ]:
teams.columns = ['full_name', 'id', 'nickname', 'city', 'state', 'year_founded']
teams.to_csv('teams.csv')

In [ ]:
# Get all players.
players = pd.DataFrame(players.get_players())

In [ ]:
players= players[players.is_active == True]

In [ ]:
#check wheter player names are unique
players.full_name.is_unique
players = players.drop('id', axis=1)

In [ ]:
players.to_csv("players.csv")


### Data Modelling 

Here we will use a datalake architecture with Amazon S3. With the schema below being defined on read. Data lakes are a secure, cost-effective and flexible approach to storing data. Though they do not inherently enforce constraints like relational databases, they make up for this through high availabity and flexible schemas.


1. Fact Table

**game_stats** - conatins unaggragated game level statistics

***columns -key, game_index, name, Date, Age, home_team_id , away_team_id, Form, GS, MP, FG, FGA, FG_pct,  three, threePA, three_pct, FT, FTA, FT_pct, ORB, DRB, TRB, AST, STL, BLK, TOV, PF, PTS, GmSc, plus_minus***

2. Dimensionals Tables

**Players** - contains player attributes

***full_name, first_name, last_name***


**Teams** - contains player attributes

***full_name, id, nickname, city, state, year_founded***



Some players exist in the fact table but not the Player table

In [ ]:
players.full_name

### Reading files to S3 

Here we are going to read the files to s3. Using Amazon's SDK `Boto3`. We are goint to use a configuration file and python's `configparser`.
 

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('lake.cfg'))

KEY     = config.get('AWS','KEY')
SECRET  = config.get('AWS','SECRET')
bucket  = config.get('S3','BUCKET')
folder  = config.get('S3', 'FOLDER')


In [ ]:
client = boto3.client('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                     )

In [ ]:
def to_s3(bucket, filename,  folder, subfolder, client):
    """
    uploads files to s3.
    - all inputs are strings
    """
    k = folder  + '/' + subfolder + filename
    client.upload_file(Filename = filename, Bucket= bucket, Key = k)

In [ ]:
## upload player_stats
to_s3(bucket, "player_stats.csv", folder, "game_stats/", client)

In [ ]:
## upload player
to_s3(bucket, "players.csv", folder, "players/", client)

In [ ]:
## upload player
to_s3(bucket, "teams.csv", folder, "teams/", client)


### Data Quality Checks
Here perform two data quality checks
1. We check if our files exist in the bucket
2. We extract the fact table and confirm rows and columns are present

#### check if our files exist in the bucket

In [ ]:
def key_existing_size_list(client, bucket, folder, subfolder, filename):
    """return the key's size if it exist, else None"""
    key = folder  + '/' + subfolder + filename
    
    response = client.list_objects_v2(
        Bucket=bucket,
        Prefix=key,
    )
    for obj in response.get('Contents', []):
        if obj['Key'] == key:
            return obj['Size']

In [ ]:
key_existing_size_list(client, bucket, folder, "game_stats/", "player_stats.csv")

In [ ]:
key_existing_size_list(client, bucket, folder, "players/", "players.csv")

In [ ]:
key_existing_size_list(client, bucket, folder, "teams/", "teams.csv")

#### check if rows and columns are present on read

In [ ]:
def row_num_s3(client, bucket, folder, subfolder, filename):
    key = folder  + '/' + subfolder + filename
    csv_obj = client.get_object(Bucket=bucket, Key=key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df  =pd.read_csv(StringIO(csv_string))
    return df.shape

In [ ]:
row_num_s3(client, bucket, folder, 'game_stats/', "player_stats.csv")

In [ ]:
row_num_s3(client, bucket, folder, 'players/', "players.csv")

In [ ]:
row_num_s3(client, bucket, folder, 'teams/', "teams.csv")

### Data dictionary

##### **Player Stats**

1. ***key*** -   Unique identifier for each entry. Player name + date 
2. ***game_index*** - incremental count of games for each player per season
3.  ***name*** - Player's full name 
4.  ***Date*** - Date when game was played
5.  ***Age*** - Player's Age
6.  ***home_team_id*** - Home team id . Abbrevated name
7.  ***away_team_id*** - Away team id. Abbrevated name
8.  ***Form*** - Oppenent form
9.  ***GS*** - If player started game. Boolean, player started game =1 
10. ***MP*** - Minutes played
11. ***FG*** - Field goals i.e Non 3 point baskets made from open play
12. ***FGA*** - Field Goal Attempts
13. ***FG_pct*** - Percentage of field goals made
14. ***three*** - Three point
15. ***threePA*** - Three point attempts
16. ***three_pct*** - Percentage of three points made
17. ***FT*** - Free throw
18. ***FTA*** - Free throw attempt
19. ***FT_pct*** - Free throw percentage
20. ***ORB*** - Offensive rebounds
21. ***DRB*** - Defensive rebounds
22. ***TRB*** - Total rebounds
23. ***AST*** - Number of assists in game.
24. ***STL*** - Number of Steals made
25. ***BLK*** - Block attempts
26. ***TOV*** - Number of turnovers
27. ***PF***  - Personal Fouls
28. ***PTS*** - Total Points
29. ***GmSc***- Game Score
30. ***plus_minus*** - plus/minus score. A measure of how impactful a player is to his team.

#### **teams**
1. ***full_name*** - Team full name
2. ***id*** - Abbrevated name
3. ***nickname*** - Team Nickname 
4. ***city*** - Team's City
5. ***state*** - Team's state
6. ***year_founded*** - Founding year

#### **players**
1. ***full_name*** - Team full name
2. ***first_name*** - Team Nickname 
3. ***last_name*** - Team's City


### Conclusion and Recommendation


#### Tools and Technologies
1. `AWS S3` for data storage
2. AWS Boto3 to access s3 client
3. Pandas to manipulate data
4. `Apache Spark` supports S3 connections therefore spark can also be used if the dataset is extended. i.e We accomodate retired players.

#### Data Update Frequency
1. The player_stats table should be played weekly to accomodate for the last matches
2. Other tables can be updated annually to accomodate any unlikely changes in values

#### Future Designs
1. The data was increased by 100x.
	
	Pandas may not be able to handle a 100x increase in data can not process 100x data set, we could consider using a spark cluster hosted `Amazon EMR`.

2. The data populates a dashboard that must be updated on a daily basis by 7am every day.

	`Apache Airflow` can be used to schedule pipeline. `Airflow` supports service level agreements that can help ensure data is uploadex in time.

3. The database needed to be accessed by 100+ people.

	We can use `Amazon Redshift`. Redshift allows for over 500 connections.
    
#### To do:
1. Automate updates
2. Add WNBA and retired players.
3. It is very likely that the scrapper will time out before it scrapes all pages for best result we can manually batch the scrapper by adding an index to `url_list`

## References
1. [Filter by regex, tutorialspoint.com](https://www.tutorialspoint.com/how-to-filter-rows-in-pandas-by-regex#:~:text=A%20regular%20expression%20%28regex%29%20is%20a%20sequence%20of,regex%2C%20we%20can%20use%20the%20str.match%20%28%29%20method.)

2. [Boto to S3](https://stackoverflow.com/questions/48399871/saving-csv-file-to-s3-using-boto3)

3. [Check s3 buckets](https://www.peterbe.com/plog/fastest-way-to-find-out-if-a-file-exists-in-s3)